In [3]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math
from datetime import datetime, timedelta
from time import strftime
import os
from requests.adapters import HTTPAdapter, Retry
import json
import os
import pandas as pd
import glob

from pbp_utils import get_matchup_games, retrieve_schedule, get_livedata_from_game, scrape_month_playbyplay
from team_utils import get_team_info, save_team_info_to_file


In [4]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2024-10-31'

In [5]:
with open('data/team_info.json', 'r') as f:
    team_info = json.load(f)

In [6]:
# doesn't include international games
start_2024 = '2024-10-08'
so_far = (retrieve_schedule(start_2024, yesterday))

In [7]:
so_far

{'game_ids': [2024020003,
  2024020004,
  2024020005,
  2024020006,
  2024020007,
  2024020008,
  2024020009,
  2024020010,
  2024020011,
  2024020012,
  2024020013,
  2024020015,
  2024020016,
  2024020014,
  2024020017,
  2024020018,
  2024020019,
  2024020020,
  2024020021,
  2024020022,
  2024020023,
  2024020024,
  2024020025,
  2024020026,
  2024020027,
  2024020028,
  2024020029,
  2024020030,
  2024020031,
  2024020032,
  2024020033,
  2024020034,
  2024020035,
  2024020036,
  2024020037,
  2024020038,
  2024020039,
  2024020040,
  2024020041,
  2024020042,
  2024020043,
  2024020044,
  2024020045,
  2024020046,
  2024020047,
  2024020048,
  2024020049,
  2024020050,
  2024020051,
  2024020052,
  2024020053,
  2024020054,
  2024020055,
  2024020056,
  2024020057,
  2024020058,
  2024020059,
  2024020060,
  2024020061,
  2024020062,
  2024020064,
  2024020065,
  2024020066,
  2024020067,
  2024020063,
  2024020068,
  2024020069,
  2024020070,
  2024020071,
  2024020072,
  202402

In [8]:
# psg_df = get_livedata_from_game(
#     {'game_ids':[2024020019],
#      'game_dates':['2024-10-10']}
# )
psg = get_livedata_from_game(so_far)

psg_df = pd.DataFrame(psg)

In [10]:
def get_player_info(df: pd.DataFrame) -> None:
    """
    Extracts unique shooter IDs from the DataFrame and calls insert_player.py to process them.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing 'shooter' column with player IDs
        
    Returns:
        None: Players are inserted into the database via insert_player.py
    """
    # Get unique shooter IDs, excluding 0 and NaN values
    unique_shooters = df['shooter'].dropna().unique()
    unique_shooters = unique_shooters[unique_shooters != 0]
    
    # Convert to integer type
    unique_shooters = unique_shooters.astype(int)
    
    try:
        from insert_player import insert_player
        
        # Process each unique shooter ID
        for player_id in unique_shooters:
            try:
                insert_player(player_id)
                print(f"Processed player ID: {player_id}")
            except Exception as e:
                print(f"Error processing player ID {player_id}: {e}")
                
    except ImportError:
        print("Could not import insert_player.py. Please ensure the file exists in your path.")
        
    print(f"Total unique shooters processed: {len(unique_shooters)}")


In [13]:
def clean_pbp_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Renames columns by removing the 'details_' prefix and performs other cleaning operations.

    Parameters:
        df (pd.DataFrame): The original DataFrame with columns to clean.

    Returns:
        pd.DataFrame: A DataFrame with cleaned columns.
    """
    # Rename columns by removing the 'details_' prefix
    df = df.rename(columns=lambda x: x.replace('details_', '') if x.startswith('details_') else x)

    # List of columns to drop if they exist
    columns_to_drop = ['maxRegulationPeriods', 'timeInPeriod', 'period_type', 'highlightClip',
                       'highlightClipFr', 'discreteClip', 'discreteClipFr', 'highlightClipSharingUrl','highlightClipSharingUrlFr']
    
    # Drop columns that exist in the DataFrame
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    # Rename columns
    column_rename_map = {
        'period_number': 'period', 'timeRemaining': 'time',
        'homeTeamDefendingSide': 'homeSide', 'typeCode': 'code',
        'typeDescKey': 'desc', 'eventOwnerTeamId': 'team', 'losingPlayerId': 'loser',
        'winningPlayerId': 'winner', 'xCoord': 'x', 'yCoord': 'y',
        'zoneCode': 'zone', 'shootingPlayerId': 'shooter', 'goalieInNetId': 'goalie',
        'hittingPlayerId': 'hitter', 'hitteePlayerId': 'hittee',
        'drawnByPlayerId': 'drawnBy', 'servedByPlayerId': 'servedBy',
        'committedByPlayerId': 'committedBy', 'blockingPlayerId': 'blocker',
        'playerId': 'player', 'scoringPlayerId': 'scorer', 'assist1PlayerId': 'assist1',
        'assist2PlayerId': 'assist2'
    }
    df = df.rename(columns={k: v for k, v in column_rename_map.items() if k in df.columns})

    # Convert player_id related columns to integers
    player_id_columns = ['player','loser', 'winner', 'hitter', 'hittee', 'shooter', 'goalie', 
                         'blocker', 'playerId', 'scorer', 'assist1',
                         'assist2', 'committedBy', 'drawnBy', 'servedBy']

    for col in player_id_columns:
        if col in df.columns:
            df[col] = df[col].fillna(0).astype(int)

    return df

In [14]:
psg_df = clean_pbp_data(psg_df)

In [15]:
get_player_info(psg_df)

Error processing player ID 8477402: insert_player() missing 1 required positional argument: 'db_config'
Error processing player ID 8479385: insert_player() missing 1 required positional argument: 'db_config'
Error processing player ID 8477416: insert_player() missing 1 required positional argument: 'db_config'
Error processing player ID 8480009: insert_player() missing 1 required positional argument: 'db_config'
Error processing player ID 8478840: insert_player() missing 1 required positional argument: 'db_config'
Error processing player ID 8479293: insert_player() missing 1 required positional argument: 'db_config'
Error processing player ID 8477986: insert_player() missing 1 required positional argument: 'db_config'
Error processing player ID 8477955: insert_player() missing 1 required positional argument: 'db_config'
Error processing player ID 8482665: insert_player() missing 1 required positional argument: 'db_config'
Error processing player ID 8477573: insert_player() missing 1 re

In [10]:
def save_pbp_month(month: int, year: int, directory: str = 'data/') -> None:
    """
    Scrapes play-by-play data for a given month and year, cleans it, and saves it to a CSV file
    with the filename format 'nhl_pbp_{month}_{year}.csv'.
    
    This function integrates the scraping and saving processes by calling the `scrape_month_playbyplay`
    function internally.
    
    Parameters:
        month (int): The month number (1-12).
        year (int): The four-digit year (e.g., 2023).
        directory (str): The directory where the file will be saved. Defaults to 'data/'.
    """
    import os
    
    # Scrape the play-by-play data for the specified month and year
    df = scrape_month_playbyplay(year, month)
    
    # Clean the scraped data
    df_cleaned = clean_pbp_data(df)
    
    # Format the filename with leading zeros for month and last two digits of year
    filename = f'nhl_pbp_{month:02d}_{str(year)[-2:]}.csv'
    file_path = os.path.join(directory, filename)
    
    # Ensure the target directory exists
    os.makedirs(directory, exist_ok=True)
    
    # Save the cleaned DataFrame to a CSV file
    df_cleaned.to_csv(file_path, index=False)
    
    print(f"Play-by-play data for {month}/{year} saved to {file_path}")

In [11]:
# Assuming you have the `scrape_month_playbyplay` and `clean_pbp_data` functions defined,
# you can save the play-by-play data for October 2023 as follows:

# save_pbp_month(month=10, year=2021)
# save_pbp_month(month=11, year=2021)
# save_pbp_month(month=12, year=2021)
# save_pbp_month(month=1, year=2022)
# save_pbp_month(month=2, year=2022)
# save_pbp_month(month=3, year=2022)
# save_pbp_month(month=4, year=2022)
# save_pbp_month(month=5, year=2022)

In [10]:
def load_pbp_files(directory='data', prefix='nhl_pbp_'):
    """
    Loads all play-by-play (PBP) CSV files in the specified directory that start with the given prefix.

    Parameters:
        directory (str): The path to the directory containing PBP files. Defaults to 'data'.
        prefix (str): The prefix that PBP files start with. Defaults to 'nhl_pbp_'.

    Returns:
        pd.DataFrame: A concatenated DataFrame containing data from all matching PBP files.
    """
    # Construct the file pattern
    pattern = os.path.join(directory, f'{prefix}*.csv')
    
    # Retrieve all file paths matching the pattern
    file_paths = glob.glob(pattern)
    
    # List to hold individual DataFrames
    df_list = []
    
    # Iterate over each file and read its contents
    for file in file_paths:
        try:
            df = pd.read_csv(file)
            df_list.append(df)
            print(f"Successfully loaded: {file}")
        except Exception as e:
            print(f"Error loading {file}: {e}")
    
    # Concatenate all DataFrames if any are loaded
    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
        print(f"Total files loaded and concatenated: {len(df_list)}")
        return combined_df
    else:
        print("No files found matching the pattern.")
        return pd.DataFrame()

In [11]:
df = load_pbp_files()
df.head(50)

Successfully loaded: data\nhl_pbp_01_24.csv
Successfully loaded: data\nhl_pbp_02_24.csv
Successfully loaded: data\nhl_pbp_03_24.csv
Successfully loaded: data\nhl_pbp_04_24.csv
Successfully loaded: data\nhl_pbp_05_24.csv
Successfully loaded: data\nhl_pbp_10_23.csv
Successfully loaded: data\nhl_pbp_11_23.csv
Successfully loaded: data\nhl_pbp_12_23.csv
Total files loaded and concatenated: 8


,gid,eventId,sortOrder,period,time,situationCode,homeSide,code,desc,team,loser,winner,x,y,zone,player,hitter,hittee,reason,shotType,shooter,goalie,awaySOG,homeSOG,blocker,scorer,scoringPlayerTotal,assist1,assist1PlayerTotal,assist2,assist2PlayerTotal,awayScore,homeScore,secondaryReason,code.1,descKey,duration,committedBy,drawnBy,servedBy
0,2023020573,102,8,1,20:00,1551.0,right,520,period-start,NaN,0,0,NaN,NaN,NaN,0,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,2023020573,101,11,1,20:00,1551.0,right,502,faceoff,55.0,8478403,8477505,0.0,0.0,N,0,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,2023020573,103,12,1,19:49,1551.0,right,504,giveaway,54.0,0,0,-98.0,4.0,D,8474565,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,2023020573,55,13,1,19:30,1551.0,right,503,hit,55.0,0,0,99.0,5.0,D,0,8479293,8475913,NaN,NaN,0,0,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,2023020573,56,15,1,19:19,1551.0,right,516,stoppage,NaN,0,0,NaN,NaN,NaN,0,0,0,icing,NaN,0,0,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
5,2023020573,105,17,1,19:19,1551.0,right,502,faceoff,55.0,8476905,8477505,69.0,22.0,D,0,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
6,2023020573,156,18,1,19:13,1551.0,right,506,shot-on-goal,54.0,0,0,81.0,-28.0,O,0,0,0,NaN,wrist,8476539,8478916,1.0,0.0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
7,2023020573,157,19,1,19:06,1551.0,right,508,blocked-shot,54.0,0,0,37.0,-6.0,D,0,0,0,blocked,NaN,8480727,0,NaN,NaN,8477955,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
8,2023020573,159,21,1,18:59,1551.0,right,506,shot-on-goal,55.0,0,0,-51.0,22.0,O,0,0,0,NaN,wrist,8479293,8480313,1.0,1.0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
9,2023020573,161,23,1,18:51,1551.0,right,508,blocked-shot,54.0,0,0,44.0,-30.0,D,0,0,0,blocked,NaN,8476539,0,NaN,NaN,8482665,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [12]:
# Filter for shot-on-goal and goal events, along with their preceding events
all_events = df.copy()

# Add a column to indicate if the **current** event is a shot or goal
all_events['is_shot_or_goal'] = all_events['desc'].isin(['shot-on-goal', 'goal'])

# Add a column to indicate if the **next** event (since data is descending) is a shot or goal
all_events['next_is_shot_or_goal'] = all_events['is_shot_or_goal'].shift(1)

# Filter for shot-on-goal and goal events, and events immediately preceding them
shot_goal_events = all_events[
    all_events['is_shot_or_goal'] | all_events['next_is_shot_or_goal']
].copy()

# Add a column to easily identify which rows are shots/goals and which are preceding events
shot_goal_events['event_type'] = np.where(
    shot_goal_events['is_shot_or_goal'],
    'shot_or_goal',
    'preceding_event'
)

# Sort the filtered data to maintain the original descending order
shot_goal_events.sort_index(inplace=True)

In [13]:
shot_goal_events.head(15)

,gid,eventId,sortOrder,period,time,situationCode,homeSide,code,desc,team,loser,winner,x,y,zone,player,hitter,hittee,reason,shotType,shooter,goalie,awaySOG,homeSOG,blocker,scorer,scoringPlayerTotal,assist1,assist1PlayerTotal,assist2,assist2PlayerTotal,awayScore,homeScore,secondaryReason,code.1,descKey,duration,committedBy,drawnBy,servedBy,is_shot_or_goal,next_is_shot_or_goal,event_type
6,2023020573,156,18,1,19:13,1551.0,right,506,shot-on-goal,54.0,0,0,81.0,-28.0,O,0,0,0,NaN,wrist,8476539,8478916,1.0,0.0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,True,False,shot_or_goal
7,2023020573,157,19,1,19:06,1551.0,right,508,blocked-shot,54.0,0,0,37.0,-6.0,D,0,0,0,blocked,NaN,8480727,0,NaN,NaN,8477955,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,False,True,preceding_event
8,2023020573,159,21,1,18:59,1551.0,right,506,shot-on-goal,55.0,0,0,-51.0,22.0,O,0,0,0,NaN,wrist,8479293,8480313,1.0,1.0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,True,False,shot_or_goal
9,2023020573,161,23,1,18:51,1551.0,right,508,blocked-shot,54.0,0,0,44.0,-30.0,D,0,0,0,blocked,NaN,8476539,0,NaN,NaN,8482665,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,False,True,preceding_event
16,2023020573,185,50,1,17:15,1551.0,right,506,shot-on-goal,54.0,0,0,54.0,-29.0,O,0,0,0,NaN,wrist,8479353,8478916,2.0,1.0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,True,False,shot_or_goal
17,2023020573,195,60,1,16:44,1551.0,right,507,missed-shot,54.0,0,0,51.0,19.0,O,0,0,0,above-crossbar,wrist,8475913,8478916,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,False,True,preceding_event
20,2023020573,216,81,1,15:31,1551.0,right,506,shot-on-goal,55.0,0,0,-47.0,26.0,O,0,0,0,NaN,wrist,8476826,8480313,2.0,2.0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,True,False,shot_or_goal
21,2023020573,60,82,1,15:29,1551.0,right,516,stoppage,NaN,0,0,NaN,NaN,NaN,0,0,0,goalie-stopped-after-sog,NaN,0,0,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,False,True,preceding_event
24,2023020573,251,85,1,15:10,1551.0,right,505,goal,55.0,0,0,-81.0,0.0,O,0,0,0,NaN,tip-in,0,8480313,NaN,NaN,0,8480009,10.0,8478407,25.0,8476457,9.0,0.0,1.0,NaN,NaN,NaN,NaN,0,0,0,True,False,shot_or_goal
25,2023020573,109,88,1,15:10,1551.0,right,502,faceoff,54.0,8481789,8479353,0.0,0.0,N,0,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,False,True,preceding_event


NameError: name 'pd' is not defined

In [141]:
import math

# Define goal location coordinates to calculate distance for shot
GOAL_LOCATION = [89, 0]

# Modify shot location coordinates
# If x coord is negative, reflect y coord so side which shot came from is consistent
df['y'] = np.where(df['x'] < 0, df['y'] * -1, df['y'])
df['x'] = np.where(df['x'] < 0, np.absolute(df['x']), df['x'])

def calculate_shot_distance(x, y):
    dist = math.dist([x, y], GOAL_LOCATION)
    return round(dist, 2)

# Calculate shot distance for shots on goal
df['shotDist'] = df.apply(lambda row: calculate_shot_distance(row['x'], row['y']) 
                          if row['desc'] == 'shot-on-goal' else None, axis=1)

# Display the first few rows to verify the new column
df[['desc', 'x', 'y', 'shotDist']].head(10)


,desc,x,y,shotDist
0,period-start,NaN,NaN,NaN
1,faceoff,0.0,0.0,NaN
2,hit,93.0,28.0,NaN
3,hit,67.0,-40.0,NaN
4,shot-on-goal,61.0,-27.0,38.90
5,missed-shot,61.0,7.0,NaN
6,shot-on-goal,79.0,-30.0,31.62
7,missed-shot,83.0,9.0,NaN
8,hit,91.0,-26.0,NaN
9,blocked-shot,73.0,-9.0,NaN
